In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels.api as sm
import pickle, time
from random import shuffle

# Define simulation functions

In [2]:
# define helper functions

# double-gamma HRF (https://github.com/poldrack/pybetaseries/blob/master/pybetaseries.py)
def spm_hrf(TR,p=[6,16,1,1,6,0,32]):
    """ An implementation of spm_hrf.m from the SPM distribution
    
    Arguments:
    
    Required:
    TR: repetition time at which to generate the HRF (in seconds)
    
    Optional:
    p: list with parameters of the two gamma functions:
                                                         defaults
                                                        (seconds)
       p[0] - delay of response (relative to onset)         6
       p[1] - delay of undershoot (relative to onset)      16
       p[2] - dispersion of response                        1
       p[3] - dispersion of undershoot                      1
       p[4] - ratio of response to undershoot               6
       p[5] - onset (seconds)                               0
       p[6] - length of kernel (seconds)                   32
    """

    p=[float(x) for x in p]

    fMRI_T = 16.0

    TR=float(TR)
    dt  = TR/fMRI_T
    u   = np.arange(p[6]/dt + 1) - p[5]/dt
    hrf=sp.stats.gamma.pdf(u,p[0]/p[2],scale=1.0/(dt/p[2])) - sp.stats.gamma.pdf(u,p[1]/p[3],scale=1.0/(dt/p[3]))/p[4]
    good_pts=np.array(range(np.int(p[6]/TR)))*fMRI_T
    hrf=hrf[list(good_pts)]
    # hrf = hrf([0:(p(7)/RT)]*fMRI_T + 1);
    hrf = hrf/np.sum(hrf);
    return hrf

# function to insert ISIs into a trial list
def insert_ISI(trials, ISI):
    return np.insert(trials, np.repeat(range(1,len(trials)), ISI), 0)

# function to build activation sequence from stimulus list
# because of how ISI is added, length of stimulus list must be a multiple of 4
# output a tidy DataFrame including
# subject info, convolved & unconvolved regressors, random effects, etc.
def build_seq(sub_num, stims, sub_A_sd, sub_B_sd):
    # shuffle stimulus list
    stims = stims.reindex(np.random.permutation(stims.index))
    
    # inter-stimulus interval is randomly selected from [1,2,3,4]
    # the first ISI is removed (so sequence begins with a stim presentation)
    ISI = np.delete(np.repeat([1,2,3,4], len(stims.index)/4, axis=0), 0)
    np.random.shuffle(ISI)
    
    # create matrix of stimulus predictors and add ISIs
    X = np.diag(stims['effect'])
    X = np.apply_along_axis(func1d=insert_ISI, axis=0, arr=X, ISI=ISI)
    
    # reorder the columns so they are in the same order (0-39) for everyone
    X = X[:,[list(stims['stim']).index([i]) for i in range(len(stims.index))]]
    
    # now convolve all predictors with double gamma HRF
    X = np.apply_along_axis(func1d=np.convolve, axis=0, arr=X, v=spm_hrf(1))
    
    # build and return this subject's dataframe
    df = pd.DataFrame(X)
    df['time'] = range(len(df.index))
    df['sub_num'] = sub_num
    # df['sub_intercept'] = np.asscalar(np.random.normal(size=1))
    df['sub_A'] = np.asscalar(np.random.normal(size=1, scale=sub_A_sd))
    df['sub_B'] = np.asscalar(np.random.normal(size=1, scale=sub_B_sd))
    return df

# the same as above, except this builds a block design experiment
def build_seq_block(sub_num, stims, sub_A_sd, sub_B_sd, block_size):
    # block stimulus list and shuffle within each block
    q = len(stims.index)
    stims = [stims.iloc[:q//2,], stims.iloc[q//2:,]]
    stims = [x.reindex(np.random.permutation(x.index)) for x in stims]
    shuffle(stims)
    stims = [[x.iloc[k:(k+block_size),] for k in range(0, q//2, block_size)] for x in stims]
    stims = pd.concat([val for pair in zip(stims[0], stims[1]) for val in pair])

    # constant ISI of 2 seconds
    ISI = np.delete(np.repeat(2, len(stims.index), axis=0), 0)

    # create matrix of stimulus predictors and add ISIs
    X = np.diag(stims['effect'])
    X = np.apply_along_axis(func1d=insert_ISI, axis=0, arr=X, ISI=ISI)

    # reorder the columns so they are in the same order (0-39) for everyone
    X = X[:,[list(stims['stim']).index([i]) for i in range(len(stims.index))]]

    # now convolve all predictors with double gamma HRF
    X = np.apply_along_axis(func1d=np.convolve, axis=0, arr=X, v=spm_hrf(1))

    # build and return this subject's dataframe
    df = pd.DataFrame(X)
    df['time'] = range(len(df.index))
    df['sub_num'] = sub_num
    # df['sub_intercept'] = np.asscalar(np.random.normal(size=1))
    df['sub_A'] = np.asscalar(np.random.normal(size=1, scale=sub_A_sd))
    df['sub_B'] = np.asscalar(np.random.normal(size=1, scale=sub_B_sd))
    return df

In [3]:
# define the main simulation function

def simulate(num_subs, num_stims, A_mean, B_mean, sub_A_sd, sub_B_sd, stim_A_sd,
    stim_B_sd, resid_sd, ar=None, block_size=None):
    # build stimulus list
    stims = np.random.normal(size=num_stims//2, loc=1, scale=stim_A_sd/A_mean).tolist() + \
            np.random.normal(size=num_stims//2, loc=1, scale=stim_B_sd/B_mean).tolist()
    stims = pd.DataFrame({'stim':range(num_stims),
                          'condition':np.repeat([0,1], num_stims//2),
                          'effect':np.array(stims)})
    
    # now build design matrix from stimulus list
    if block_size is None:
        # build event-related design
        data = pd.concat([build_seq(sub_num=i, stims=stims, sub_A_sd=sub_A_sd, sub_B_sd=sub_B_sd) for i in range(num_subs)])
    else:
        # build blocked design
        data = pd.concat([build_seq_block(sub_num=i, stims=stims, sub_A_sd=sub_A_sd, sub_B_sd=sub_B_sd, block_size=block_size) for i in range(num_subs)])

    # add response variable and difference predictor
    if ar is None:
        # build y WITHOUT AR(2) errors
        data['y'] = (A_mean + data['sub_A'])*data.iloc[:,:(num_stims//2)].sum(axis=1).values + \
                    (B_mean + data['sub_B'])*data.iloc[:,(num_stims//2):num_stims].sum(axis=1).values + \
                    np.random.normal(size=len(data.index), scale=resid_sd)
    else:
        # build y WITH AR(2) errors
        data['y'] = np.empty(len(data.index))
        data['y_t-1'] = np.zeros(len(data.index))
        data['y_t-2'] = np.zeros(len(data.index))
        for t in range(len(pd.unique(data['time']))):
            data.loc[t,'y'] = pd.DataFrame(
                (A_mean + data.loc[t,'sub_A'])*data.loc[t, range(num_stims//2)].sum(axis=1).values + \
                (B_mean + data.loc[t,'sub_B'])*data.loc[t, range(num_stims//2, num_stims)].sum(axis=1).values + \
                np.random.normal(size=len(data.loc[t].index), scale=resid_sd)).values
            if t==1:
                data.loc[t,'y'] = pd.DataFrame(data.loc[t,'y'].values + ar[0]*data.loc[t-1,'y'].values).values
                data.loc[t,'y_t-1'] = pd.DataFrame(data.loc[t-1,'y']).values
            if t>1:
                data.loc[t,'y'] = pd.DataFrame(data.loc[t,'y'].values + ar[0]*data.loc[t-1,'y'].values + ar[1]*data.loc[t-2,'y'].values).values
                data.loc[t,'y_t-1'] = pd.DataFrame(data.loc[t-1,'y']).values
                data.loc[t,'y_t-2'] = pd.DataFrame(data.loc[t-2,'y']).values

    # remove random stimulus effects from regressors before fitting model
    data.iloc[:, :num_stims] = data.iloc[:, :num_stims] / stims['effect'].tolist()

    ####################################
    ############ FIT SPM MODEL #########
    ####################################
    
    # fit subject-level regressions and return difference in slopes
    def get_diff(df):
        X = pd.concat([df.iloc[:,:num_stims//2].sum(axis=1),
                       df.iloc[:,num_stims//2:num_stims].sum(axis=1),
                       df['y_t-1'],
                       df['y_t-2']], axis=1)
        beta = pd.stats.api.ols(y=df['y'], x=X, intercept=False).beta
        return pd.Series(beta[1] - beta[0]).append(beta)
    sub_diffs = data.groupby('sub_num').apply(get_diff).iloc[:,0]

    # one-sample t-test on the difference scores
    t_test = pd.stats.api.ols(y=sub_diffs,
                              x=pd.Series(np.repeat(1,len(sub_diffs))),
                              intercept=False)
    return np.asscalar(t_test.p_value)

# Run the simulation

In [11]:
# build the full parameter list

r = np.arange(500) # number of iterations in each cell
n = m = np.array([16,32,64]) # n, m = number of participants, stimuli
s = np.array([.00001, 1, 2]) # SD of random stimulus effects
params = np.meshgrid(r, n, m, s)
params = pd.DataFrame({x: params[i].flatten() for i,x in enumerate(['r','n','m','s'])})
params.head()

,m,n,r,s
0,16,16,0,0.00001
1,16,16,0,1.00000
2,16,16,0,2.00000
3,32,16,0,0.00001
4,32,16,0,1.00000


In [12]:
# run simulation

start = time.clock()
params['p_value'] = [
    simulate(num_subs=int(params.loc[i,'n']), num_stims=int(params.loc[i,'m']), A_mean=1,
             B_mean=1, sub_A_sd=1, sub_B_sd=1, stim_A_sd=params.loc[i,'s'],
             stim_B_sd=params.loc[i,'s'], resid_sd=1, ar=[.45,.15], block_size=8)
    for i in params.index]
end = time.clock()

//anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:34: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future
//anaconda/envs/python3/lib/python3.5/site-packages/pandas/core/groupby.py:647: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://statsmodels.sourceforge.net/stable/regression.html
  return func(g, *args, **kwargs)
//anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://statsmodels.sourceforge.net/stable/regression.html


In [13]:
# time elapsed (in hours)
(end - start)/60/60

11.2499852825

# Summarize and store results

In [14]:
params['alpha05'] = params['p_value'] < .05
params['alpha01'] = params['p_value'] < .01
params['alpha005'] = params['p_value'] < .005
params['alpha001'] = params['p_value'] < .001
params.groupby(['s','n','m'])[['alpha05','alpha01','alpha005','alpha001']].mean()

alpha05  alpha01  alpha005  alpha001
s       n  m                                       
0.00001 16 16    0.068    0.014     0.004     0.002
           32    0.054    0.014     0.002     0.002
           64    0.044    0.006     0.006     0.000
        32 16    0.060    0.012     0.010     0.000
           32    0.060    0.006     0.004     0.000
           64    0.062    0.012     0.008     0.002
        64 16    0.052    0.020     0.010     0.004
           32    0.056    0.014     0.010     0.000
           64    0.044    0.008     0.002     0.000
1.00000 16 16    0.178    0.058     0.036     0.008
           32    0.128    0.056     0.036     0.010
           64    0.118    0.038     0.028     0.010
        32 16    0.262    0.158     0.124     0.062
           32    0.224    0.102     0.080     0.036
           64    0.130    0.042     0.024     0.008
        64 16    0.442    0.312     0.260     0.182
           32    0.342    0.196     0.142     0.072
           64    0.248    0.130     0.102     0.050
2.00000 16 16    0.344    0.190     0.148     0.082
           32    0.274    0.150     0.106     0.046
           64    0.208    0.104     0.080     0.030
        32 16    0.478    0.316     0.270     0.200
           32    0.438    0.322     0.262     0.170
           64    0.290    0.186     0.148     0.080
        64 16    0.642    0.548     0.514     0.410
           32    0.568    0.442     0.400     0.310
           64    0.452    0.298     0.254     0.184

In [15]:
# store results
params.to_json('xsim_false_positive_results.json')